In [1]:
import numpy as np
import pandas as pd
import pylab as plt
import random as rd

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.util import minibatch, compounding
from nltk.stem.porter import *  

import re
import urllib.request
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = None
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

nlp = spacy.load("en_core_web_md")

from spacy import displacy
import df_helper as dfh
import nlp_support as ns


from sklearn.utils.random import sample_without_replacement
import importlib
import pickle
from sklearn.feature_selection import SelectKBest, chi2,  RFE, RFECV, SelectFpr

In [2]:
importlib.reload(dfh)
importlib.reload(ns)

<module 'nlp_support' from '/home/marco/UD/NLP_disaster/nlp_support.py'>

In [3]:
from sklearn.base import TransformerMixin


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, validation_curve
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, confusion_matrix

import sklearn.metrics as met
from sklearn.metrics import precision_recall_curve, roc_curve, auc, matthews_corrcoef
from sklearn.metrics import confusion_matrix, fbeta_score, make_scorer, average_precision_score, auc, \
    accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, classification_report, \
    brier_score_loss, roc_auc_score

from scipy.stats import randint as sp_randint , uniform
import eli5

from joblib import dump, load


In [6]:
#%%timeit -n1 -r1

if 1:   
    
    X_train,Y_train,X_test, Y_test,X_valid, Y_valid= ns.load_data()
    n_false_sample= 2500
    n_features=100

    DOE='under_false_%s_nwords_%s_mindf_1' % (n_false_sample,n_features)
    print(DOE)

    data_set, bow_bal, tfidf = ns.train_bow(X_train,Y_train, X_test,Y_test,n_false_sample ,  max_df=0.9,min_df=1, max_features=n_features )

    with open('%s_data.pkl' % DOE, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data_set, f)
        
        from joblib import dump, load
        dump(tfidf, '%s_tfidf.joblib' % DOE)
        dump(bow_bal, '%s_bal_cv.joblib' % DOE)
        
    print('Saved')

/home/marco/MLE/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


under_false_2500_nwords_100_mindf_1
Original n_true, n_false: 2329 18717
Resampled n_true, n_false: 2329 2500
Saved


In [23]:
words_bal=set(bow_bal.get_feature_names())


In [4]:
# with open('%s_data.pkl' % DOE, 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(data_set, f)

#     from joblib import dump, load
#     dump(tfidf, '%s_tfidf.joblib' % DOE)
#     dump(bow_bal, '%s_bal_cv.joblib' % DOE)

# print('Saved')

NameError: name 'DOE' is not defined

In [11]:

if 1:

    X_train,Y_train,X_test, Y_test,X_valid, Y_valid= ns.load_data()
    n_false_sample= None
    n_features=5000

    DOE='fs_false_all_nwords_%s_mindf_1' % (n_features)
    print(DOE)

    data_set2, bow_bal2, tfidf2 = ns.train_bow(X_train,Y_train, X_test,Y_test,n_false_sample ,  max_df=0.9,min_df=1, max_features=n_features )

    with open('%s_data.pkl' % DOE, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data_set2, f)
        
        from joblib import dump, load
        dump(tfidf2, '%s_tfidf.joblib' % DOE)
        dump(bow_bal2, '%s_bal_cv.joblib' % DOE)
        
    print('Saved')

fs_false_all_nwords_5000_mindf_1
Original n_true, n_false: 2329 18717
setting n_false_sample to:  18717
Resampled n_true, n_false: 2329 18717
Saved


In [14]:
fs=SelectKBest(chi2, k=100)



In [19]:
fs.fit_transform(data_set2['Xtrain'], data_set2['Ytrain'])

<21046x100 sparse matrix of type '<class 'numpy.float64'>'
	with 24421 stored elements in Compressed Sparse Row format>

In [22]:
sup=fs.get_support()
words_all=np.array(bow_bal2.get_feature_names())
words_fs=set(words_all[sup])

In [34]:
print("common words in FS and in balanced:\n")

common_words=np.array( words_bal.intersection(words_fs) )
display(common_words)

common words in FS and in balanced:



array({'aid', 'hunger', 'item', 'bring', 'drought', 'donat', 'medic', 'wfp', 'distribut', 'help', 'medicin', 'product', 'peopl', 'inform', 'cloth', 'water', 'sleep', 'die', 'street', 'need', 'eat', 'hungri', 'tent', 'hygien', 'shelter', 'suppli', 'haiti', 'rice', 'earthquak', 'babi', 'blanket', 'food'},
      dtype=object)

In [33]:
print("words in FS, not in balanced:\n")
display( np.array(words_fs - words_bal) )

words in FS, not in balanced:



array({'potabl', 'sandwich', 'nt', 'brach', 'feuill', 'hurri', 'intersect', 'drinkabl', 'tonn', 'thirst', 'meal', 'clercin', 'famin', 'thomazeau', 'starv', 'clean', 'flour', 'eaten', 'jacket', 'kachipul', 'pack', 'impass', 'noodl', 'toy', 'biscuit', 'thirsti', 'oil', 'parcel', 'canapevert', 'bread', 'arcahai', 'feed', 'etc', 'nutriti', 'bean', 'insecur', 'jacmel', 'non', 'bag', 'rue', 'shortag', 'ton', 'leogan', 'instant', 'carrefour', 'nourish', 'box', 'hot', 'kilo', 'nonperish', 'will', 'avenu', 'nutrit', 'champagn', 'note', 'ration', 'packet', 'section', 'deliv', 'fort', 'toiletri', 'frere', 'gressier', 'laboul', 'perish', 'cook', 'can', 'delma'},
      dtype=object)

In [36]:
print("words in balanced, not in FS:\n")
display( np.array(words_bal - words_fs) )

words in balanced, not in FS:



array({'affect', 'year', 'like', 'support', 'destroy', 'come', 'live', 'health', 'caus', 'school', 'region', 'heavi', 'work', 'week', 'home', 'day', 'famili', 'nation', 'commun', 'build', 'citi', 'storm', 'said', 'problem', 'power', 'emerg', 'provinc', 'good', 'messag', 'ask', 'govern', 'disast', 'countri', 'diseas', 'continu', 'children', 'area', 'district', 'flood', 'villag', 'includ', 'find', 'state', 'rain', 'assist', 'place', 'relief', 'sandi', 'world', 'unit', 'road', 'report', 'crop', 'time', 'hurrican', 'intern', 'thank', 'know', 'victim', 'receiv', 'provid', 'hous', 'local', 'damag', 'send', 'secur', 'want', 'kill'},
      dtype=object)

In [ ]:
if 1:   
    
    X_train,Y_train,X_test, Y_test,X_valid, Y_valid= ns.load_data()
    n_false_sample= 0
    n_features=1000

    DOE='only_false_%s_nwords_%s_mindf_1' % (n_false_sample,n_features)
    print(DOE)

    data_set3, bow_bal3, tfidf = ns.train_bow(X_train,Y_train, X_test,Y_test,n_false_sample ,  max_df=0.9,min_df=1, max_features=n_features )

    with open('%s_data.pkl' % DOE, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data_set, f)
        
        from joblib import dump, load
        dump(tfidf, '%s_tfidf.joblib' % DOE)
        dump(bow_bal3, '%s_bal_cv.joblib' % DOE)
        
    print('Saved')

only_false_0_nwords_1000_mindf_1
Original n_true, n_false: 2329 18717
Resampled n_true, n_false: 2329 0


In [ ]:
fs=SelectKBest(chi2, k=100)
fs.fit_transform(data_set3['Xtrain'], data_set3['Ytrain'])

sup=fs.get_support()
words_all2=np.array(bow_bal3.get_feature_names())
words_fs2=set(words_all2[sup])
